In [446]:
import pandas as pd
import altair as alt
import math

xls = pd.ExcelFile(
    "../PART II/data/Political Tweet data.xlsx"
)

data = pd.read_excel( xls, "misleading data" )
chart = alt.Chart(data)

In [447]:
class color:

    hex_values = [ "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F"  ]

    def __init__(self, R, G, B):
        self.R = R
        self.G = G
        self.B = B

    def return_color_between(self, color2, perc):
        r_change = color2.R - self.R
        g_change = color2.G - self.G
        b_change = color2.B - self.B

        r = self.R + (r_change * perc)
        g = self.G + (g_change * perc)
        b = self.B + (b_change * perc)

        return color(r, g, b)

    def return_color_in(self, code):
        if code == "RGB":
            return "{} ({}, {}, {})".format(code, self.R * 255, self.G * 255, self.B * 255)
        if code == "HSB":
            return "{} ({}, {}, {})".format(code, self.R * 360, self.G * 100, self.B * 100)
        if code == "HEX":
            hex1 = self.return_hex(self.R)
            hex2 = self.return_hex(self.G)
            hex3 = self.return_hex(self.B)
            return "#{}{}{}{}{}{}".format( hex1[0], hex1[1], hex2[0], hex2[1], hex3[0], hex3[1] )

    def return_hex(self, component ):
        rounded = math.floor(component / 16)
        remainder = (component / 16) - rounded

        hex1 = self.hex_values[rounded]
        hex2 = self.hex_values[round(remainder * 16)]
        return (hex1, hex2)

    def return_color_grad(self, second_color, steps):
        colors = []
        for step in range(0, steps):
            interval = step / (steps - 1)
            color = self.return_color_between(second_color, interval)
            colors.append(color.return_color_in("HEX"))
        return colors



In [448]:
class pallet:
    def __init__(self, grad, primary_color, secondary_color, background, secondary_background):
        self.grad = grad
        self.primary_color = primary_color
        self.secondary_color = secondary_color
        self.background = background 
        self.secondary_background = secondary_background

color2 = color( 120, 38, 72 )
color1 = color( 164, 109, 168 )

grad = color1.return_color_grad( color2, 7 )

prim = color2.return_color_in("HEX")
sec =  color(148, 52, 92).return_color_in("HEX")
back = color(242, 225, 206).return_color_in("HEX")
sec_back = color(204, 185, 163).return_color_in("HEX")

pallet = pallet( grad, prim, sec, back, sec_back  )

In [449]:

def apply_bar_styling(chart, pallet):
    return chart.configure_bar(
        cornerRadius=5
    ).configure_range (
        category=pallet.grad
    )

def apply_text(chart, pallet):
    return chart.configure(
        font="DINCondensed-Bold",
        background=pallet.background
    ).configure_axis(
        labelFontSize=13,
        titleFontSize=25,
        labelColor=pallet.secondary_color,
        titleColor=pallet.primary_color,
        domainColor=pallet.secondary_background,
        tickColor=pallet.secondary_background,
        gridColor=pallet.secondary_background
    ).configure_title(
        fontSize=30,
        color=pallet.primary_color 
    )

def apply_legend(chart, pallet):
    return chart.configure_legend(
        cornerRadius=10,
        fillColor=pallet.secondary_background,
        labelColor=pallet.primary_color,
        labelFontSize=13,
        padding=10,
        titleColor=pallet.primary_color,
        titleFontSize=15


    )

def apply_all(chart, pallet):
    tmp_chart = apply_text(chart, pallet)
    tmp_chart = apply_legend(tmp_chart, pallet)
    tmp_chart = apply_bar_styling(tmp_chart, pallet)
    return tmp_chart

In [450]:
unequal = chart.mark_bar().encode (
    alt.X( "bias-trun", title="Target Audience" ),
    alt.Y( "value", title="Number of Tweets" ),
    alt.Color( "category")
).properties(
    title="The categorical breakdown of different target audiences"
)


apply_all(unequal, pallet)

alt.Chart(...)

In [451]:
equal = chart.mark_bar().encode(
    alt.X( "bias-trun", title="target Audience" ),
    alt.Y( "perc:Q", title="number of tweets in a category over total tweets"),
    alt.Color( "category" )
).properties(
    title="The categorical breakdown of different target audiences, normalized"
)

apply_all(equal, pallet)

alt.Chart(...)

In [452]:
unequal_misleading = chart.mark_bar().encode (
    alt.X( "bias-trun", title="target Audience" ),
    alt.Y( "invert", title="Number of tweets in a category, inverted" ),
    alt.Color( "category" )
).properties(
    title="The inverted categorical breakdown of different target audiences"
)

apply_all(unequal_misleading, pallet)

alt.Chart(...)

In [453]:
equal_misleading = chart.mark_bar().encode (
    alt.X( "bias-trun", title="target Audience"  ),
    alt.Y( "invert-perc", title="inverted number of tweets in a category over total tweets"),
    alt.Color( "category" )
).properties(
    title="The inverted categorical breakdown of different target audiences, normalized"
)

apply_all(equal_misleading, pallet)

alt.Chart(...)